In [3]:
library("LOLA")
library("GenomicRanges")
library("data.table")

Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors

Attaching package: ‘S4Vectors’

The followin

##  Enrichment for CREM0109128

In [24]:
c_regionDB = loadRegionDB("../../../data/references/LOLA/LOLACore/hg38/")
c_activeDHS = readBed("../data/TCGA/BRCA/all_rems.bed")
c_regionA = readBed("../data/TCGA/BRCA/CREM0109128.bed")

Reading collection annotations: 
	cistrome_cistrome: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/cistrome_cistrome/collection.txt
	cistrome_epigenome: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/cistrome_epigenome/collection.txt
	codex: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/codex/collection.txt
	encode_tfbs: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/encode_tfbs/collection.txt
	sheffield_dnase: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/sheffield_dnase/collection.txt
	ucsc_features: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/ucsc_features/collection.txt
Reading region annotations...
::Loading cache::	../../../data/references/LOLA/LOLACore/hg38/cistrome_cistrome//cistrome_cistrome_files.RData
::Loading cache::	../../../data/references/LOLA/LOLACore/hg38/cistrome_epigenome//cistrome_epigenome_files.RData
::Loading

In [25]:
c_userSets = GRangesList(c_regionA)
c_locResults = runLOLA(c_userSets, c_activeDHS, c_regionDB, cores=1)

Calculating unit set overlaps...
Calculating universe set overlaps...
Calculating Fisher scores...


In [30]:
writeCombinedEnrichment(c_locResults, outFolder= "../data/TCGA/BRCA/", includeSplits=TRUE)

Overwriting ../data/TCGA/BRCA/col_cistrome_cistrome.tsv...
Overwriting ../data/TCGA/BRCA/col_cistrome_epigenome.tsv...
Overwriting ../data/TCGA/BRCA/col_codex.tsv...
Overwriting ../data/TCGA/BRCA/col_encode_tfbs.tsv...
Overwriting ../data/TCGA/BRCA/col_sheffield_dnase.tsv...
Overwriting ../data/TCGA/BRCA/col_ucsc_features.tsv...
Overwriting ../data/TCGA/BRCA/allEnrichments.tsv...


## Enrichment for DysRegNet results

In [53]:
# collect bed files and read 
files <- list.files(path = "../data/TCGA/BRCA/bed_files/", pattern = "\\.bed$", full.names =TRUE)

data <- lapply(files, readBed)

regionDB = loadRegionDB("../../../data/references/LOLA/LOLACore/hg38/")
activeDHS = readBed("../data/TCGA/BRCA/dys_rems_BRCA.bed")
userSets = GRangesList(data)

# run LOLA
locResults = runLOLA(userSets, activeDHS, regionDB, cores=1)

# write to files
writeCombinedEnrichment(locResults, outFolder= "../data/TCGA/BRCA/lolaResults/", includeSplits=TRUE)

Reading collection annotations: 
	cistrome_cistrome: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/cistrome_cistrome/collection.txt
	cistrome_epigenome: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/cistrome_epigenome/collection.txt
	codex: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/codex/collection.txt
	encode_tfbs: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/encode_tfbs/collection.txt
	sheffield_dnase: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/sheffield_dnase/collection.txt
	ucsc_features: found collection annotation:../../../data/references/LOLA/LOLACore/hg38/ucsc_features/collection.txt
Reading region annotations...
::Loading cache::	../../../data/references/LOLA/LOLACore/hg38/cistrome_cistrome//cistrome_cistrome_files.RData
::Loading cache::	../../../data/references/LOLA/LOLACore/hg38/cistrome_epigenome//cistrome_epigenome_files.RData
::Loading

## Analyse results

In [19]:
lola_list <- list.files(path = "../data/TCGA/BRCA/lolaResults/", pattern = "\\.tsv$", full.names =TRUE)
file_samples <- list()
for (i in files){
    file_samples <- append(file_samples, strsplit(strsplit(i, "/")[[1]][7], "_")[[1]][1])
    }

In [6]:
lola_files = list()
for(i in 1: 746) {
    lola_files <- append(lola_files, sprintf('../data/TCGA/BRCA/lolaResults//userSet_%s.tsv', i))
    }

In [21]:
sub <- read.csv("../data/TCGA/BRCA/samples_subtypes_filt.csv",
                header = TRUE, sep = "\t")
sub <- as.data.table(sub[sub$sample %in% file_samples, ])
sub <- sub[ ,(c("sample", "subtype")) := lapply(.SD, as.character), .SDcols = c("sample", "subtype")]
x <- make.unique(c(unlist(file_samples)))
sub <- sub[match(x, make.unique(unlist(sub$sample)))]

In [9]:
tfs_odd <- list()
tfs_mxRank <- list()
tfs_meanRank <- list()
for (f in lola_files){
    dt <- as.data.table(readr::read_tsv(f))
    
    dt_odd <- dt[order(oddsRatio, decreasing=FALSE),]
    tfs_odd <- append(tfs_odd, na.omit(dt_odd$antibody)[1])
    
    dtmax <- dt[order(maxRnk, decreasing=FALSE),]
    tfs_mxRank <- append(tfs_mxRank, na.omit(dtmax$antibody)[1])
    
    dtmean <- dt[order(meanRnk, decreasing=FALSE),]
    tfs_meanRank <- append(tfs_meanRank, na.omit(dtmean$antibody)[1])
    }

Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve 

dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (8): collection, description, cellType, tissue, antibody, treatment, da...
dbl (14): userSet, dbSet, pValueLog, oddsRatio, support, rnkPV, rnkOR, rnkSu...
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.Rows: 2969 Columns: 22── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [348]:
tfs <- tfs_mxRank
subt <- unlist(sub$subtype)
Am <- matrix(0, nrow=length(unique(tfs)),ncol=length(unique(subt)))
dimnames(Am) <- list(unique(tfs), unique(subt))

for( i in seq_along(tfs)) {
#     print(i)
    Am[tfs[[i]],subt[i]] <- Am[tfs[[i]],subt[i]] + 1
}

In [349]:
write.table(as.data.frame(Am),file="../data/TCGA/BRCA/lola_matrix_maxRank.csv")

## Visualization

In [ ]:
library(ggplot2)
library(ggpubr)
library(dplyr)

In [25]:
mat <- read.csv("../data/TCGA/BRCA/lola_matrix_maxRank.csv", header = TRUE, sep = " ")
mat_filt <- mat %>% filter(if_any(where(is.numeric), ~. > 2))

mat_mean <- read.csv("../data/TCGA/BRCA/lola_matrix_meanRank.csv", header = TRUE, sep = " ")
mat_filt_mean <- mat_mean %>% filter(if_any(where(is.numeric), ~. > 1))

In [ ]:
ggballoonplot(mat_filt, fill = "value", ggtheme = xlab("Subtype"))+
    scale_fill_viridis_c("Count",option = "C") +
    guides(size = FALSE) +
    theme_bw() + ggtitle("Number of samples enriched for Antibody/TFB in subtype") +
    xlab("Subtype") + ylab("Antibody/TFB")

In [ ]:
ggsave("../data/TCGA/BRCA/lola_matrix_filt_max.png", height = 17, width = 8)